<a href="https://colab.research.google.com/github/chandini2595/CMPE255_Assignment2/blob/main/Autogluon_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other require

In [2]:
!pip install autogluon.multimodal[torch]==0.7.0  # Adjust version as needed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 25.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [4]:
import os
import zipfile
import requests

# Download the dataset
dataset_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_file = 'cats_and_dogs_filtered.zip'

# Download and save the dataset
response = requests.get(dataset_url, stream=True)
with open(zip_file, 'wb') as f:
    f.write(response.content)

# Extract the dataset
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("cats_and_dogs_filtered")

In [5]:
import pandas as pd

# Path to the extracted dataset
dataset_dir = 'cats_and_dogs_filtered'

# Create a DataFrame with image file paths and their corresponding labels
image_paths = []
labels = []

# Cats folder
cats_dir = os.path.join(dataset_dir, 'cats_and_dogs_filtered/train/cats')
for img_file in os.listdir(cats_dir):
    image_paths.append(os.path.join(cats_dir, img_file))
    labels.append('cat')

# Dogs folder
dogs_dir = os.path.join(dataset_dir, 'cats_and_dogs_filtered/train/dogs')
for img_file in os.listdir(dogs_dir):
    image_paths.append(os.path.join(dogs_dir, img_file))
    labels.append('dog')

# Create DataFrame
train_df = pd.DataFrame({
    'image': image_paths,
    'label': labels
})

# Display the first few rows of the DataFrame
print(train_df.head())

                                               image label
0  cats_and_dogs_filtered/cats_and_dogs_filtered/...   cat
1  cats_and_dogs_filtered/cats_and_dogs_filtered/...   cat
2  cats_and_dogs_filtered/cats_and_dogs_filtered/...   cat
3  cats_and_dogs_filtered/cats_and_dogs_filtered/...   cat
4  cats_and_dogs_filtered/cats_and_dogs_filtered/...   cat


In [8]:
from autogluon.multimodal import MultiModalPredictor
from autogluon.core.utils.loaders import load_pd
import pandas as pd
import zipfile
import requests
import os


# Create a MultiModalPredictor, specifying the label column
predictor = MultiModalPredictor(label='label')

# Train the model on the images, using GPU (if available)
predictor.fit(
    train_data=train_df,
    time_limit=600,  # Limit training to 10 minutes
    presets='best_quality'
    # Use high-quality presets
)

# Save the trained model
predictor.save('cats_vs_dogs_model')

INFO:lightning_fabric.utilities.seed:Global seed set to 123
No path specified. Models will be saved in: "AutogluonModels/ag-20240923_210316/"
AutoMM starts to create your model. ✨

- Model will be saved to "/content/AutogluonModels/ag-20240923_210316".

- Validation metric is "roc_auc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20240923_210316
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_pat

Sanity Checking: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 6: 'val_roc_auc' reached 0.99697 (best 0.99697), saving model to '/content/AutogluonModels/ag-20240923_210316/epoch=0-step=6.ckpt' as top 3


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 13: 'val_roc_auc' reached 0.99977 (best 0.99977), saving model to '/content/AutogluonModels/ag-20240923_210316/epoch=0-step=13.ckpt' as top 3
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Start to fuse 2 checkpoints via the greedy soup algorithm.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 13>:13                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/autogluon/multimodal/predictor.py:825 in fit             │
│                                                                                                  │
│    822 │   │   │   )                                                                             │
│    823 │   │   │   return predictor                                                              │
│    824 │   │                                                                                     │
│ ❱  825 │   │   self._fit(**_fit_args)                                                            │
│    826 │   │   training_end = time.time()                                                        │
│    827 │   │   self._total_train_time = training_end - training_start                            │
│    828                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/autogluon/multimodal/predictor.py:1395 in _fit           │
│                                                                                                  │
│   1392 │   │   │   # We do not perform averaging checkpoint in the case of hpo for each trial    │
│   1393 │   │   │   # We only averaging the checkpoint of the best trial in the end in the maste  │
│   1394 │   │   │   if not hpo_mode:                                                              │
│ ❱ 1395 │   │   │   │   self._top_k_average(                                                      │
│   1396 │   │   │   │   │   model=model,                                                          │
│   1397 │   │   │   │   │   save_path=save_path,                                                  │
│   1398 │   │   │   │   │   minmax_mode=minmax_mode,                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/autogluon/multimodal/predictor.py:1474 in _top_k_average │
│                                                                                                  │
│   1471 │   │   │   │   │   │   │   f"Start to fuse {len(top_k_model_paths)} checkpoints via the  │
│   1472 │   │   │   │   │   │   )                                                                 │
│   1473 │   │   │   │   │   │                                                                     │
│ ❱ 1474 │   │   │   │   │   │   self._model = self._load_state_dict(                              │
│   1475 │   │   │   │   │   │   │   model=model,                                                  │
│   1476 │   │   │   │   │   │   │   path=top_k_model_paths[0],                                    │
│   1477 │   │   │   │   │   │   │   prefix=prefix,                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/autogluon/multimodal/predictor.py:2319 in                │
│ _load_state_dict                                                                                 │
│                                                                                                  │
│   2316 │   │   strict: bool = True,                                                              │
│   2317 │   ):                                                                                    │
│   2318 │   │   if state_dict is None:                                                            │
│ ❱ 2319 │   │   │   state_dict = torch.load(path, map_locati

In [9]:
import torch
print("GPU Available:", torch.cuda.is_available())

GPU Available: True
